### IS Proof of concept
Finding Product position on Social Network (PPSN)

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import json
import pymongo

In [2]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
C:\ProgramData\Anaconda3\lib\site-packages\_pytest\mark\structures.py:378: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


In [3]:
import time
from datetime import datetime
today = datetime.now()

In [4]:
#!pip install pythainlp
import pythainlp
from pythainlp import Tokenizer
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp import sent_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import dict_trie
from pythainlp.tag import pos_tag

In [5]:
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# https://thainlp.org/pythainlp/docs/2.0/api/tag.html
from pythainlp.spell import correct
from pythainlp.tag import pos_tag

#https://thainlp.org/pythainlp/docs/2.0/api/transliterate.html
from pythainlp.transliterate import romanize

In [7]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [8]:
from IPython.display import clear_output

In [9]:
# Files definition
root_path = 'Datasource'

comment_file = []
comment_file.append(root_path+"/comment_meiji.csv")
comment_file.append(root_path+"/comment_DDCF.csv")

#comment_file = []
#comment_file.append("comment_meiji.csv")
#comment_file.append("comment_DDCF.csv")

comment_nlp_file = root_path+"/comment_nlpToken.csv"
comment_cooc_nlp_file = root_path+"/comment_cooc.xlsx"

#comment_nlp_file = "comment_nlpToken.csv"
#comment_cooc_nlp_file = "comment_cooc.xlsx"

In [10]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["NIDA_PPSN"]
col_thread = mydb["NIDA_PPSN_THREAD_V2"]
col_comment = mydb["NIDA_PPSN_COMMENT"]

In [11]:
def checkStopwords(w):
    returnList = []
    stopwords = set(thai_stopwords())
    stopwords.remove('ไม่')
    stopwords.remove('ได้')
    for i in w:  # comment
        if i not in stopwords:
            returnList.append(i)
    return returnList

In [12]:
def checkThai(txt):
    prop = pythainlp.util.countthai(txt)
    return prop

In [13]:
def normThai(w):
    returnList = []
    for i in w:
        returnList.append(normalize(i))
    return returnList

In [14]:
def tagThai(w):
    pd_returnList = pd.DataFrame(pos_tag(w, corpus='pud'))
    returnList = pd_returnList.iloc[:,1]
    return returnList

In [15]:
def soundThai(w):
    returnList = []
    for i in w:
        returnList.append(romanize(i, engine="thai2rom"))
    return returnList

In [16]:
list_punc = []
for c in string.punctuation:
    list_punc.append(c)

### Clean data & NLP Tokenized

In [17]:
df_textList1 = pd.read_csv(comment_file[0],header=None)
df_textList2 = pd.read_csv(comment_file[1],header=None)

In [18]:
df_textList1.insert(0,'Table','Meiji')
df_textList1.columns = ['Table','URLs', 'text', 'Retrived_date']
df_textList2.insert(0,'Table','DDFC')
df_textList2.columns = ['Table','URLs', 'text', 'Retrived_date']

In [19]:
pd.DataFrame(df_textList1.URLs.value_counts()).describe().T

,count,mean,std,min,25%,50%,75%,max
URLs,1000.0,43.049,53.682105,3.0,11.0,21.0,49.0,351.0


In [20]:
print('meiji comment:',df_textList1.shape[0])
print('meiji URLs ก่อนตัด:',len(df_textList1.URLs.unique()))

meiji comment: 43049
meiji URLs ก่อนตัด: 1000


In [21]:
pd.DataFrame(df_textList2.URLs.value_counts()).describe().T

,count,mean,std,min,25%,50%,75%,max
URLs,517.0,44.756286,57.056773,1.0,10.0,19.0,53.0,328.0


In [22]:
print('DDCF comment:',df_textList2.shape[0])
print('DDCF URLs ก่อนตัด:',len(df_textList2.URLs.unique()))

DDCF comment: 23139
DDCF URLs ก่อนตัด: 517


In [23]:
df_Corpus = pd.concat([df_textList1,df_textList2],ignore_index=True)

In [24]:
print('All comment:',df_Corpus.shape[0])
print('All URLs ก่อนตัด:',len(df_Corpus.URLs.unique()))

All comment: 66188
All URLs ก่อนตัด: 1517


In [25]:
missing = df_Corpus.isnull().sum()
print(missing)

Table               0
URLs                0
text             4703
Retrived_date       0
dtype: int64


In [26]:
df_Corpus.tail()

,Table,URLs,text,Retrived_date
66183,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930
66184,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930
66185,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930
66186,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930
66187,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930


In [27]:
seqNum = list(range(1,df_Corpus.shape[0]+1))
df_Corpus.insert(0,'commentId',seqNum)
df_Corpus.columns = ['commentId','Table','URLs', 'text', 'Retrived_date']

In [28]:
df_Corpus.tail()

,commentId,Table,URLs,text,Retrived_date
66183,66184,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930
66184,66185,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930
66185,66186,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930
66186,66187,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930
66187,66188,DDFC,https://pantip.com/topic/30651064,NaN,2020-10-27 13:54:33.920930


In [29]:
# Drop NA & banned comments
df_Corpus = df_Corpus.dropna()
df_Corpus = df_Corpus[df_Corpus["text"].str.find('ความคิดเห็นนี้ถูกลบ')!= 0]
df_Corpus = df_Corpus[df_Corpus["text"].str.find('แก้ไขข้อความเมื่อ')!= 0]
df_Corpus = df_Corpus.set_index('commentId')

In [30]:
missing = df_Corpus.isnull().sum()
print(missing)

Table            0
URLs             0
text             0
Retrived_date    0
dtype: int64


In [31]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 60545
จำนวนกระทู้หลังตัด: 1514


In [32]:
df_Corpus.insert(4,'token_text',None)
#df_Corpus.insert(5,'token_sound',None)
#df_Corpus.insert(6,'token_tag',None)
df_Corpus.tail()

,Table,URLs,text,Retrived_date,token_text
commentId,,,,,
66181,DDFC,https://pantip.com/topic/30651064,นมผง คือเอานมดิบดีๆ มาคัดไขมันนมและอื่นๆออกไปใ...,2020-10-27 13:54:33.920930,None
66182,DDFC,https://pantip.com/topic/30651064,นมสด รสชาติมันดีนะ\nเพราะมีปริมาณไขมันสูง ดังน...,2020-10-27 13:54:33.920930,None
66184,DDFC,https://pantip.com/topic/30651064,มีหลายยี่ห้อนะค่ะ พวกจิตรดา ไทยเดนมาร์ค โฟร์โม...,2020-10-27 13:54:33.920930,None
66185,DDFC,https://pantip.com/topic/30651064,เดี๋ยวนมรสจืด ส่วนใหญ่ก็นมโคแท้ 100%ทั้งนั้นแห...,2020-10-27 13:54:33.920930,None
66186,DDFC,https://pantip.com/topic/30651064,ตามกฏหมายเอง นมโคสดแท้ 100% กับนมโคแท้ 100% ไม...,2020-10-27 13:54:33.920930,None


เก็บเฉพาะหัวกระทู้

In [33]:
df_thread = df_Corpus.copy()
df_thread = df_thread[['URLs','text']]
df_thread.drop_duplicates(subset ="URLs", keep = 'first', inplace = True)
df_thread.insert(2,'token_text',None)
df_thread.insert(3,'mention_product',None)
df_thread.insert(4,'mention_brand',None)
df_thread.tail()

,URLs,text,token_text,mention_product,mention_brand
commentId,,,,,
65991,https://pantip.com/topic/30007179,กำลังจะเป็นคุณแม่มือใหม่ค่ะ ทำตัวไม่ถูกเลยค่ะ\...,None,None,None
66015,https://pantip.com/topic/30020409,พรุ่งนี้ลูกชายอายุครบ 1 ขวบ 3 เดือน หม่ำนมแม่จ...,None,None,None
66044,https://pantip.com/topic/30049123,"เพื่อนๆทุกคนคงคุ้นเคยกับคำถามนี้แล้วทุกคน... ""...",None,None,None
66074,https://pantip.com/topic/30011023,แตกกระทู้มาจาก www.pantip.com/topic/30007146/\...,None,None,None
66132,https://pantip.com/topic/30651064,อยากรู้ว่าปัจจุบันพวกนมกล่องทึ่ขายใน 7-11 หรือ...,None,None,None


## ตัดคำและเก็บลง MongoDB (ใช้ insight จาก jupyter งานก่อน)

https://www.thainlp.org/pythainlp/tutorials/notebooks/pythainlp-get-started.html#Thai-Characters

In [34]:
garbage_char = ['',' ','🥺','\r','\n','\r\n','\xa0','www','application','ทางกฎหมาย','html'
                ,'http','https','url','com','pantip','topic','_','–','พันทิป'
                ,'นะค่ะ','นะคะ','น่ะค่ะ','ฯ','ๆ','แฮ่','เย่ๆๆ','ว๊าว','ฮิ้วววววววววว','Number','กก'
                ,'เมนู','Menu','Net','net','สาขา','บาท','ราคา','(',')','ฯ','ๆ','กก','อันนี้','%s','😆', '🤣','😢'
                ,'😏','😂','😿','ววว','xx','อิอิ','แย้ววว','นะคะ','นะค่ะ','://','มกราคม', 'กุมภาพันธ์'
                ,'มีนาคม', 'เมษายน','พฤษภาคม', 'มิถุนายน', 'กรกฎาคม', 'สิงหาคม','กันยายน', 'ตุลาคม'
                ,'พฤศจิกายน', 'ธันวาคม']
garbage_char  = garbage_char + list_punc

In [35]:
print('จำนวน comment หลังตัด:',df_Corpus.shape[0])
print('จำนวนกระทู้หลังตัด:',len(df_Corpus.URLs.unique()))

จำนวน comment หลังตัด: 60545
จำนวนกระทู้หลังตัด: 1514


In [36]:
custom_list = ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่','เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji'
               ,'โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟโมต','โชคชัย','แดรี่โฮม','dairy home','อืมม มิลค์','Umm! Milk','เอ็มมิลค์'
               ,'mmilk','เอ็มมิ้ลค์','แมคโนเลีย','นมฟาร์มโชคชัย','นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง','หนองโพ','คาเนชั่น'
               ,'สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่'
               ,'ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','มิ้นท์ชอค','มิ้นต์ช้อก','รสกล้วย'
               ,'รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','low fat','0% fat','ขาดมันเนย','พร่องมันเนย','ไฮแคลเซียม','high protein'
               ,'ไฮโปรตีน','เวย์โปรตีน','whey','เวย์','อัลมอนด์','ซีเล็ค','ซีเล็คท์','กล้วยหอม','ไวท์คอฟฟี่','คาปูชิโน่','อเมริกาโน่'
               ,'grass fed','free lactose','lactose free','พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์'
               ,'เมจิโกลด์ แม็กซ์','เมจิโกลด์'
               ,'น้ำตาลมะพร้าว','นมฮอกไกโด','เบดไทม์','ฟรีแลคโตส','แลคโตสฟรี','ริชเอสเพสโซ่','ดาร์คช็อกโกแลต'
               ,'คาราเมล','ช็อกโกมอลต์','เมล่อน','ชาเขียวมัจฉะ','ฝาน้ำเงิน'
               ,'ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','แมคโคร','แม็คโคร','โลตัส','บิ๊กซี','bigc','Golden place','Big C'
               ,'ขายไม่ดี','แพคคู่','ค่าจัดส่ง','shelf life','พนักงานขายนม'
               ,'แพ้นม','แพ้นมวัว','เล่นเวท','นำ้หนัก','คุณแม่มือใหม่','นมอุ่น','ชานม','กินนม','ดื่มนม','ท้องเสีย','ขี้แตก'
               ,'คีเฟอร์นม','พันทิป','ร้านนม','ซื้อไม่ได้','ไม่ได้','ลูกสุนัข','ลูกหมา'
              ]
words = set(thai_words()).union(set(custom_list))
custom_tokenizer = Tokenizer(engine='newmm', custom_dict=words, keep_whitespace=False)

In [37]:
def find_brands_product(x,catType):
    ret_wtkn = []
    brands = ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่','เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji'
               ,'โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟโมต','โชคชัย','แดรี่โฮม','dairy home','อืมม มิลค์','Umm! Milk','เอ็มมิลค์'
               ,'mmilk','เอ็มมิ้ลค์','แมคโนเลีย','นมฟาร์มโชคชัย','นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง','หนองโพ','คาเนชั่น']
    products = ['สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่','นม'
               ,'ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต','มิ้นท์ชอค','มิ้นต์ช้อก','รสกล้วย'
               ,'รสหวาน','รสจืด','ไขมันต่ำ','ไขมัน 0%','low fat','0% fat','ขาดมันเนย','พร่องมันเนย','ไฮแคลเซียม','high protein'
               ,'ไฮโปรตีน','เวย์โปรตีน','whey','เวย์','อัลมอนด์','ซีเล็ค','ซีเล็คท์','กล้วยหอม','ไวท์คอฟฟี่','คาปูชิโน่','อเมริกาโน่'
               ,'grass fed','free lactose','lactose free','พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์','เมจิโกลด์ แม็กซ์'
               ,'น้ำตาลมะพร้าว','นมฮอกไกโด','เบดไทม์','ฟรีแลคโตส','แลคโตสฟรี','ริชเอสเพสโซ่','ดาร์คช็อกโกแลต'
               ,'คาราเมล','ช็อกโกมอลต์','เมล่อน','ชาเขียวมัจฉะ','ฝาน้ำเงิน','เมจิโกลด์ แม็กซ์','เมจิโกลด์']
    
    choiceList = []
    if catType == 'b':
        choiceList = brands
    else:
        choiceList = products
        
    for i,each in enumerate(x,0):
        if each in choiceList:
            ret_wtkn.append(each)
    return ret_wtkn

In [38]:
def create_InsertMany_Thread(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "URLs": lx,
            "text": df.text[c],
            "token_text":df.token_text[c],
            "mention_product": df.mention_product[c],
            "mention_brand": df.mention_brand[c]
        }
        listofdict.append(info)
    return listofdict

In [39]:
def create_InsertMany_comments(df):
    listofdict = []
    for c,lx in enumerate(df.URLs,0):
        info = {
            "commentId":int(df.commentId[c]),
            "Table": df.Table[c],
            "URLs": lx,
            "text": df.text[c],
            "Retrived_date": df.Retrived_date[c],
            "token_text":df.token_text[c]
        }
        listofdict.append(info)
    return listofdict

In [40]:
def tokenize_custom(x,custom_tokenizer):
    wtkn = custom_tokenizer.word_tokenize(x)
    wtkn = checkStopwords(wtkn)
    wtkn = [x for x in wtkn if len(x) >= 2]
    wtkn = [y for y in wtkn if y not in garbage_char]
    wtkn = [s for s in wtkn if not s.isdigit()]
    wtkn = normThai(wtkn)
    return wtkn

In [41]:
def generalize_keyword(wtkn):  
    ret_wtkn = wtkn
    for i,each in enumerate(ret_wtkn,0):
        if each in ['ดัชมิลล์','ดัชมิล','dutch mill','dutchmilk','ดัชชี่']:
            ret_wtkn[i] = 'ดัชมิลล์'
        elif each  in ['เมจิ','ซีพีเมจิ','ซีพี-เมจิ','cp-meiji','meiji','meji','miji']:
            ret_wtkn[i] = 'เมจิ'
        elif each  in ['โฟร์โมสต์','โฟร์โมส','โฟโมสต์','โฟโมต']:
            ret_wtkn[i] = 'โฟร์โมสต์'
        elif each  in ['โชคชัย','อืมม มิลค์','Umm! Milk','นมฟาร์มโชคชัย']:
            ret_wtkn[i] = 'โชคชัย'
        elif each  in ['เอ็มมิลค์','mmilk','เอ็มมิ้ลค์']:
            ret_wtkn[i] = 'เอ็มมิลค์'
        elif each  in ['นมไทย-เดนมาร์ค','ไทยเดนมาร์ค','วัวแดง']:
            ret_wtkn[i] = 'ไทยเดนมาร์ค'
        elif each  in ['สตอเบอร์รี่','สตรอว์เบอร์รี','รสสตรอเบอรี่','สตรอเบอรี่','สตรอวเบอรี่']:
            ret_wtkn[i] = 'สตรอว์เบอร์รี'
        elif each  in ['ชอคโกแล็ต','ช็อกโกแลต','ช็อคโกแลต','ช๊อกโกแลต','ช็อคโกแล็ต','ช็อกโกเลต']:
            ret_wtkn[i] = 'ช็อกโกแลต'
        elif each  in ['มิ้นท์ชอค','มิ้นต์ช้อก','ช็อคโกแลตมินต์','มินต์ช็อคโกแลต']:
            ret_wtkn[i] = 'ช็อกโกแลตมินต์'
        elif each  in ['ไขมันต่ำ','low fat','พร่องมันเนย']:
            ret_wtkn[i] = 'ไขมันต่ำ'
        elif each  in ['ไขมัน 0%','0% fat','ขาดมันเนย']:
            ret_wtkn[i] = 'ไขมัน 0%'
        elif each  in ['high protein','ไฮโปรตีน']:
            ret_wtkn[i] = 'ไฮโปรตีน'
        elif each  in ['free lactose','lactose free','ฟรีแลคโตส','แลคโตสฟรี']:
            ret_wtkn[i] = 'นมฟรีแลคโตส'
        elif each  in ['พาสเจอไรซ์','พาสเจอร์ไรซ์','พาสเจอร์ไรส์','พาซเจอไรซ์','พาสเจอร์ไลท์']:
            ret_wtkn[i] = 'พาสเจอร์ไรส์'
        elif each  in ['เมจิโกลด์ แม็กซ์','เมจิโกลด์']:
            ret_wtkn[i] = 'เมจิโกลด์'
        elif each  in ['ฝาน้ำเงิน','รสจืด']:
            ret_wtkn[i] = 'รสจืด'
        elif each  in ['ท็อปส์','ทอปส์','ท้อปส์','ท๊อปส์','tops']:
            ret_wtkn[i] = 'tops'
        elif each  in ['แมคโคร','แม็คโคร','makro']:
            ret_wtkn[i] = 'makro'
        elif each  in ['โลตัส','lotus']:
            ret_wtkn[i] = 'lotus'
        elif each  in ['บิ๊กซี','bigc','Big C']:
            ret_wtkn[i] = 'bigc'
        elif each  in ['ท้องเสีย','ขี้แตก']:
            ret_wtkn[i] = 'ท้องเสีย'
        elif each  in ['ลูกสุนัข','ลูกหมา']:
            ret_wtkn[i] = 'ลูกสุนัข'
    return ret_wtkn

#### Copy ชุดคำจาก File มา Process ก่อนลง MongoDB

In [42]:
df_process_comment = df_Corpus.reset_index().copy()
df_process_thread = df_thread.reset_index().copy()

1.Tokenize comment

In [43]:
df_process_comment['token_text'] = df_process_comment['text'].apply(lambda x: tokenize_custom(x,custom_tokenizer))

In [44]:
df_process_comment.head()

,commentId,Table,URLs,text,Retrived_date,token_text
0,1,Meiji,https://pantip.com/topic/39868603,เมื่อก่อนหลายปีมาแล้ว ในเซเว่นยังมีนมเมจิรสหวา...,2020-10-26 12:40:26.135083,"[ปี, เซเว่น, ยังมี, นม, เมจิ, รสหวาน, ขาย, อยู..."
1,2,Meiji,https://pantip.com/topic/39868603,เบาหวานครับ แฮ่ๆ,2020-10-26 12:40:26.135083,[เบาหวาน]
2,3,Meiji,https://pantip.com/topic/39868603,หาซื้อไม่ได้นี่คิดได้ 2 อย่างนะ. คนไม่ค่อย...,2020-10-26 12:40:26.135083,"[หา, ซื้อไม่ได้, คิดได้, , . คน, กินกัน, ไม..."
3,4,Meiji,https://pantip.com/topic/39868603,เป็นรสที่ขายไม่ดีครับ\r\nรสพื้นฐานคือ\r\nจืด แ...,2020-10-26 12:40:26.135083,"[รส, ขายไม่ดี, รส, พื้นฐาน, จืด, แบ่ง, %,, low..."
4,5,Meiji,https://pantip.com/topic/39868603,ถ้าขายไม่ดีเค้าก็เอาออกครับ shelf life มันน้อย...,2020-10-26 12:40:26.135083,"[ขายไม่ดี, เค้า, shelf life, เวลา, หมดอายุ, ค่..."


2.Generalized keyword (Brand & product)

In [45]:
df_process_comment['token_text'] = df_process_comment['token_text'].apply(lambda x: generalize_keyword(x))

3.Insert comments to DB

In [46]:
col_comment.delete_many({})
process_data = create_InsertMany_comments(df_process_comment)
col_comment.insert_many(process_data)

4.Tokenize head of thread

In [47]:
df_process_thread['token_text'] = df_process_thread['text'].apply(lambda x: tokenize_custom(x,custom_tokenizer))

5.Generalized keyword (Brand & product)

In [48]:
df_process_thread['token_text'] = df_process_thread['token_text'].apply(lambda x: generalize_keyword(x))

6.Mark domain/entity mention

In [49]:
df_process_thread['mention_brand'] = df_process_thread['token_text'].apply(lambda x: find_brands_product(x,'b'))
df_process_thread['mention_product'] = df_process_thread['token_text'].apply(lambda x: find_brands_product(x,'p'))

In [50]:
df_process_thread.tail()

,commentId,URLs,text,token_text,mention_product,mention_brand
1509,65991,https://pantip.com/topic/30007179,กำลังจะเป็นคุณแม่มือใหม่ค่ะ ทำตัวไม่ถูกเลยค่ะ\...,"[คุณแม่มือใหม่, ทำตัว, ไม่, ทราบ, ที่จะ, กินนม...",[],[]
1510,66015,https://pantip.com/topic/30020409,พรุ่งนี้ลูกชายอายุครบ 1 ขวบ 3 เดือน หม่ำนมแม่จ...,"[พรุ่งนี้, ลูกชาย, อายุ, ขวบ, เดือน, หม่ำ, นม,...",[นม],[]
1511,66044,https://pantip.com/topic/30049123,"เพื่อนๆทุกคนคงคุ้นเคยกับคำถามนี้แล้วทุกคน... ""...","[เพื่อน, คุ้นเคย, คำถาม, ทุ, กค, น., .., ?"", ห...",[],[]
1512,66074,https://pantip.com/topic/30011023,แตกกระทู้มาจาก www.pantip.com/topic/30007146/\...,"[แตก, กระทู้, มาจาก, บ้าน, รูป, เฟส, นะจ๊ะ, วั...",[],[]
1513,66132,https://pantip.com/topic/30651064,อยากรู้ว่าปัจจุบันพวกนมกล่องทึ่ขายใน 7-11 หรือ...,"[อยากรู้, นมกล่อง, ทึ่, ขาย, ห้างสรรพสินค้า, น...",[นม],[]


7.Insert thread to DB

In [51]:
col_thread.delete_many({})
process_data = create_InsertMany_Thread(df_process_thread)
col_thread.insert_many(process_data)

In [52]:
del df_textList1,df_textList2,df_thread,df_Corpus,df_process_comment,df_process_thread

### Process หัวกระทู้เพื่อดึง Domain (Brand & Product) ไม่ใช้โมเดลเทรนแล้วทำนาย ต้องการความถูกต้อง 100%

หากเคยบันทึกลง MongoDB แล้ว ให้เริ่มตรงนี้

In [53]:
cursor_thread = col_thread.find()
df_thr_process = pd.DataFrame(cursor_thread)

In [54]:
df_urls_milk = df_thr_process[(df_thr_process.mention_brand.str.len() != 0) & (df_thr_process.mention_product.str.len() != 0)]

In [55]:
df_urls_milk.shape

(414, 6)

In [56]:
#df_urls_milk.to_excel("Docs/วิเคราะห์ข้อมูล/domain.xlsx",index=False)

#### คัดเลือกกระทู้ที่มีความเกี่ยวข้อง แล้วดึงคำไปใช้

In [57]:
cursor_comment = col_comment.find()
df_cmt_process = pd.DataFrame(cursor_comment)

In [58]:
df_join_url = pd.merge(df_cmt_process,df_urls_milk,how='inner',on='URLs')
df_join_url = df_join_url[['Retrived_date','URLs','commentId','mention_brand', 'mention_product','token_text_x']]
df_join_url.columns = ['Retrived_date','URLs','commentId','mention_brand', 'mention_product','token_text']

In [59]:
df_cmt_process.shape

(60545, 7)

In [60]:
df_join_url.tail()

,Retrived_date,URLs,commentId,mention_brand,mention_product,token_text
7351,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65703,[โฟร์โมสต์],"[นม, รสจืด]","[ปกติ, ขับ, ออกจาก, แยก, ไม่ได้, เลี้ยว, ไม่, ..."
7352,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65704,[โฟร์โมสต์],"[นม, รสจืด]","[เสียใจ, รถ, ถนน, , ไม่, , ระวัง, ตัวเอง, ดีกว..."
7353,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65705,[โฟร์โมสต์],"[นม, รสจืด]","[หน้า, ห้าง, ..., วาง, กรวย, หา, แพะ]"
7354,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65706,[โฟร์โมสต์],"[นม, รสจืด]","[ขี่, รถ, ผ่านหน้า, bigc, รัชดา, ผม, วิ่ง, เลน..."
7355,2020-10-27 13:51:00.344890,https://pantip.com/topic/30153058,65707,[โฟร์โมสต์],"[นม, รสจืด]","[ชีวิต, เลือก, ได้, ไม่, ขับ, มอไซด์, เสี่ยง, ..."


In [61]:
df_view = df_join_url[['commentId','mention_brand']]
df_view['mention_brand'] = df_join_url['mention_brand'].apply(lambda x: repr(set(x)))
df_view.column = ['commentId','mention_brand']

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [62]:
df_view.groupby('mention_brand').count()

,commentId
mention_brand,
"{'คาเนชั่น', 'ดัชมิลล์'}",12
{'คาเนชั่น'},19
"{'ดัชมิลล์', 'หนองโพ', 'ไทยเดนมาร์ค'}",7
"{'ดัชมิลล์', 'เอ็มมิลค์', 'เมจิ'}",2
"{'ดัชมิลล์', 'ไทยเดนมาร์ค'}",13
{'ดัชมิลล์'},1076
"{'หนองโพ', 'ดัชมิลล์'}",47
"{'หนองโพ', 'เมจิ'}",41
"{'หนองโพ', 'โชคชัย', 'ดัชมิลล์'}",26


## สร้าง Bag of word dataframe
ไม่มีทางเลือก Review Lit มาแล้ว ต้องคัดคำระดับ 1000 คำ มาทำ รันไปเลย 4-6 ชั่วโมง รันจบแล้วถึงทำ  Factor Analysis/T-SNE ต่อ<br>
ให้คัดเลือก flavor, brand, market clean ตาม term ให้ครบ นอกนั้นให้ดึงมาทั้งหมดตรงๆ

1. ตัดคำ และ Process คำ รอบที่ 2
2. Clean คำ ชื่อ Flavor และชื่อ Brand
3. นับคำในแต่ละเอกสาร (Comment)
4. รอ รอ รอ

### Create co-occurrence matrix จาก Bag of word

### Create co-occurrence matrix with Lift normalization